In [ ]:
import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
file1=os.path.expanduser('~/Documents/log/2019-01-02T18-36-25/logstream_Ser.bin')
file2=os.path.expanduser('~/Documents/log/2019-01-03T09-10-12/logstream_Ser.bin')

In [ ]:
# Create a dtype with the binary data format and the desired column names
dt = np.dtype([('idx', '>u1'),
               ('cmdSec', '>u4'), ('cmdUsec', '>u4'), ('cmd', '>u4'),
               ('fbSec', '>u4'), ('fbUsec', '>u4'),
               
               ('length', '>u2'),
               ('counts', '>u2'),
               ('voltage', '>u2'),
               ('temperature', '>u2'),
               ('current', '>u2'),
               ('id', 'u1')])

In [ ]:
data1 = np.fromfile(file1, dtype=dt)
df1 = pd.DataFrame(data1)

data2 = np.fromfile(file2, dtype=dt)
df2 = pd.DataFrame(data2)

In [ ]:
df1=df1.loc[(df1['id'] == 171) & (df1['idx'] == 0)]
df2=df2.loc[(df2['id'] == 171) & (df2['idx'] == 0)]

In [ ]:
plt.plot(df1.counts)

In [ ]:
plt.plot(df2.counts)